<a href="https://colab.research.google.com/github/DeKUT-DSAIL/DSA-2024-NLP/blob/main/pre-lab/prompt_engineering_prelab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DSA 2024 - NLP Lab Session

### Part 3: Prompt Engineering with LLaMA-2

Prompt engineering is the discipline of developing and optimising prompts to effectively use large language models (LLMs) to achieve desired outputs for a wide variety of applications, including research. By developing prompt engineering skills, we are enabled to better understand the capabilities and limitations of these LLMs.

The key aspects of prompt engineering include, but are not limited to:
* Crafting clear prompts: The model's output is significantly affected by the model's output. To get accurate and relevant responses, prompts should be clear, concise, and specific.
* Providing context: Prompts that include sufficient context within them help the models understand the background and generate more informed responses. Contexts can involve giving background information, setting the scene, or specifying the desired format of the answer.
* Iterative refinement: Prompt engineering is often an iterative process where initial prompts are continuously adjusted and refined to improve the quality of the response.
* Instruction precision: Explicity stating what you want from the model can dramatically improve outcomes. Using words like "list", "describe", etc. help guide the model more effectively.
* Balancing length and detail: Although detailed prompts can provide more guidance, overly long prompts tend to confuse the model. Striking a balance between providing enough details and maintaining brevity is important.
* Leveraging special tokens: Some models allow the use of special tokens or specific structures to control responses, such as separators or format indicators. `LLaMA-2` is one such model.

#### Prerequisites
This lab is targeted at Python developers who have some familiarity with LLMs, such as by using ChatGPT or Gemini, but have limited experience in working with LLMs in a programmatic way.

If you're familiar with the underpinnings of LLMs, you'll have a slight advantage. However, familiarity with basic Python and a basic understanding of LLMs will be sufficient to help you get a lot out of this course.

For this lab, we shall be working with the `LLaMA-2` model available at [HuggingFace](https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ). In order to download this model for use in this notebook, you will need to install the [transformers](https://pypi.org/project/transformers/) package. Not to worry, the steps for installing it are baked into this Colab notebook and you will not need to take any extra steps, except if you choose to run the notebook locally.


#### Learning Objectives
1. Use a `transformers` pipeline to generate responses from a LLaMA-2 LLM.
2. Iteratively write precise prompts to get the desired output from the LLM.
3. Work with the LLaMA-2 prompt template to perform instruction fine-tuning.
4. Use LLaMA-2 to generate JSON data for potential use in downstream processing tasks


Let's get cracking.

### Setup

Let's get started by setting up our environment by installing the `transformers` package and importing the necessary packages.

Remember to change the runtime type on Colab to GPU by following these steps:
* On the menu bar, click on `Runtime`.
* Click `change runtime type`.
* In the `Hardware accelerator` radio options, select `T4 GPU`.
* Click `save`.



In [ ]:
! pip install transformers
! pip install accelerate
! pip install optimum
! pip install auto-gptq

In [ ]:
from transformers import pipeline
import accelerate

Let's ignore the warrnings to keep the cell outputs nice and clean

In [ ]:
import warnings
warnings.filterwarnings("ignore")

def warn(*args, **kwargs):
    pass
warnings.warn = warn

### Create LLaMA-2 Pipeline

In [ ]:
model = "TheBloke/Llama-2-13B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model, device_map="auto")

#### A Helper Function

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    This function takes a prompt and passes it to the model e.g. LLaMA and returnss the response from the model

    Parameters:
    @param prompt (str): The input text prompt to generate a response for.
    @param max_length (int): The maximum length, in tokens, of the generated response.
    @param pipe (callable): The model's pipeline function used for generation.
    @param **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

**First prompt: Capital of California**

We shall begin with a very simple prompt and pass it to the `LLaMA-2` model. The desired outcome is that the model responds to us with only the name of the capital of California, which is *Sacramento*, with nothing else in the response.

In [ ]:
prompt = "What is the capital of California?"

print(generate(prompt))

The model responds back with additional helpful information about the city of Sacramento. We are not interested in this additional information. We want the name of the city and no additional context, so let's craft a prompt that is more **specific**.

In [ ]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible."

print(generate(prompt))

Answer: Sacramento


That was a better response, but we still get a leading `Answer:` in the reponse. We can prevent this behaviour by providing the model with the **cue** `Answer:`. Doing this can prevent the model from providing the text itself.

In [ ]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible. Answer: "

print(generate(prompt))

Sacramento.


**Second prompt: Vowels in 'Sacramento'**

In this part of the notebook, we ask the model to do somethig more complicated i.e., tell us the vowes found in the name of the capital of California.

We know the correct answer is S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo**. Notice that, in order to arrive at the correct answer, you probably had to perform the task in multiple steps.

In [ ]:
prompt = "Tell me the vowels in the capital of California."

print(generate(prompt))

When LLMs are faced with the need to reason in a way that requires multiple steps, it is often helpful to craft a prompt instructing the model to perform multiple intermediary steps, like asking it to show its working. This technique is often described as giving the model **"time to think"**.

Let's now craft a new prompt asking the model to take the intermediate step of identifying the capital of Kenya before identifying the vowels in it.

In [ ]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it."

print(generate(prompt))

We can see that giving the model **time to think** made a big difference. Armed with this new technique, let's ask the model to do something more complicated - tell the vowels in the name of the capital of Kenya in reverse alphabetical order.

The correct answer is S**a**cr**a**m**e**nt**o** -> **aeo** -> **oea**

In [ ]:
prompt = "Tell me the vowels in the capital of California in reverse alphabetical order?"

print(generate(prompt))

Obviously, we did not give the model **time to think**. Let's ask it to break the task down to intermediate steps and show its work.

In [ ]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it, then tell me the vowels in reverse-alphabetical order."

print(generate(prompt))

#### Exercise
Ask the model to perform the product of 37 and 48 and then iteratively develop a concise prompt which results in the correct response from the model.

**Note**: LLMs aren't the best tools for performing math. So, be sure to consider how you can be **precise** in your prompt and also allow the model **time to think**.

If you get stuck, we have provided a solution further down.

In [ ]:
# The right answer
37*48

1776

In [ ]:
prompt = "37x48"

print(generate(prompt))

#### Your work here

#### Solution

<details>
<summary>
Click me to see the solution
</summary>

```python
prompt = "Calculate the product of 37 and 48. Use the steps typical of long multiplication and show your work."
print(generate(prompt))
```
</details>

## Review

In this notebook, you have learnt the following key concepts:

* **Precise**: Being as specific and explicit as necessary to guide the response of an LLM.
* **Cue**: A phrase to conclude your prompt to guides its response, and often to prevent it from including the cue itself in its response.
* **"Time to think"**: A quality in prompts that supports LLM responses (often requiring calculation) by asking for the model to take multiple steps and show its work.